In [ ]:
import numpy as np
from scripts.Markovian import Markovian
from pathlib import Path

In [ ]:
PATH = Path().parent.resolve()

# PRESSURE_LEVELS_VALUES = [925, 850, 700, 500, 300, 200, 100]
PRESSURE_LEVELS_VALUES = [925]
N_LAT = 73
N_LON = 144

VAR = ['air', 'uwnd', 'vwnd', 'rhum']
VAR_STD = dict({
    'air': 1,
    'uwnd': 1,
    'vwnd': 1,
    'rhum': 0.0001,
})

VAR_DATA_TRAIN = np.load(PATH/'data/atmos_cond_train.npz')
VAR_DATA_TEST = np.load(PATH/'data/atmos_cond_test.npz')

In [ ]:
var_models = dict({
    var: Markovian(N_LAT, N_LON, n_jobs=-1).fit(vals)
    for var, vals in VAR_DATA_TRAIN.items()})

In [ ]:
STEPS, _ = VAR_DATA_TEST['air'].shape

In [ ]:
da_results = dict({
    var: model.EnKFMC_results(
        steps=STEPS,
        std=VAR_STD[var],
        p_obs=0.05,
        r=7,
        atms_ens=VAR_DATA_TRAIN[var],
        atms_obs=VAR_DATA_TEST[var],
        seed=10)
    for var, model in var_models.items()
})